In [1]:
!pip install tensorflow-addons
!pip install git+https://www.github.com/keras-team/keras-contrib
!pip install shap
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/master thesis/code/xai-tsc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.1/612.1 kB 4.7 MB/s eta 0:00:00
  Cloning https://www.github.com/keras-team/keras-contrib to /tmp/pip-req-build-kbkckll4
  Running command git clone --filter=blob:none --quiet https://www.github.com/keras-team/keras-contrib /tmp/pip-req-build-kbkckll4
  Resolved https://www.github.com/keras-team/keras-contrib to commit 3fc5ef709e061416f4bc8a92ca3750c824b5d2b0
  Preparing metadata (setup.py) ... done
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-py3-none-any.whl size=101056 sha256=524c6d1bbb813c085dbdc80dbbee2f6111b4b977bf77a725e2b596c8f4e2ba7e
  Stored in directory: /tmp/pip-ephem-wheel-cache-2_j1o9yi/wheels/f9/d5/fe/883253892023a0ee62cc0a8d57dc9ae22635258dca2c510ec0
Successfully built keras-contrib
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.9/547.9 kB 6.3 MB/s eta 0:00:00
Mounted at /content/drive
/content/drive/My Drive/master thesis/code/xai-tsc


In [88]:
import shap
from utils.utils import read_dataset
from tensorflow import keras
import tensorflow  as tf
import numpy as np

from utils.explanations import minmax_norm, norm

archive = 'ucr'
dataset_name = 'GunPoint'
data_p = 'G:/Meine Ablage/master thesis/code/xai-tsc'
dataset = 'GunPoint'
model_type ='fcn'
ylen = 1
itr = 0
scale = 'normalized'




task = 0

x_train,x_test,y_train,y_test = read_dataset(data_p, archive, dataset, 'original', ylen)[dataset]
#Load model 
model_link = f'./results/ucr/{dataset}/experiment_1/{model_type}/{model_type}_{itr}/original/best_model.hdf5'
loaded_model = keras.models.load_model(model_link ,compile=False)
#get random background data sample
background_data = np.array(x_train[np.random.randint(0,x_train.shape[0],10)])
#init Permutation Sample
e = shap.PermutationExplainer(loaded_model,background_data, nsamples=100)
#Calculate Shap values for trian data
#>>>train
train_shap_val = np.array(e.shap_values(x_train))
#>>>test
test_shap_val = np.array(e.shap_values(y_train))
#store output 1 > train_data , 2 > test_data
#get format yval,xval,att
output = []

att = train_shap_val if ii == 0 else test_shap_val
for x_vals,y_vals in [[x_train,x_test],[y_train,y_test]]:
    pred = loaded_model.predict(x_vals) if task == 0 else loaded_model.predict(x_vals)[0]
    attr = list()
    print(len(x_vals))
    for idx,ts in enumerate(x_vals):
        if scale == 'minmax': 
            # get shap values for predicted class > minmax scaling
            shap_att = minmax_norm(att[idx,:, np.argmax(pred[idx])])
        if scale == 'normalized': 
            # get shap valuues for predicted class > L2 normalized
            shap_att = norm(att[idx,:,np.argmax(pred[idx])])
        attr.append([y_vals[idx],x_vals[idx],shap_att])
    output.append(attr)

shap_vals = calcualte_shap_values()

Permutation explainer:  30%|███       | 15/50 [01:01<02:59,  5.13s/it]


KeyboardInterrupt: 

In [79]:
#get format yval,xval,att
output = []

att = train_shap_val if ii == 0 else test_shap_val
for x_vals,y_vals in [[x_train,x_test],[y_train,y_test]]:
    pred = loaded_model.predict(x_vals) if task == 0 else loaded_model.predict(x_vals)[0]
    attr = list()
    print(len(x_vals))
    for idx,ts in enumerate(x_vals):
        if scale == 'minmax': 
            # get shap values for predicted class > minmax scaling
            shap_att = minmax_norm(att[idx,:, np.argmax(pred[idx])])
        if scale == 'normalized': 
            # get shap valuues for predicted class > L2 normalized
            shap_att = norm(att[idx,:,np.argmax(pred[idx])])
        attr.append([y_vals[idx],x_vals[idx],shap_att])
    output.append(attr)



2/2 [==============================] - 0s 6ms/step
50
5/5 [==============================] - 0s 9ms/step
150


In [86]:
from utils.explanations import create_explanations, save_explanations

expl = create_explanations(output)
save_explanations(expl, data_p, archive, 'shap', dataset_name)

(50, 300) (150, 300)
Successfully created explanation done.
